In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
MODEL_NAME = "bert-base-uncased"

MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
LEARNING_RATE = 1e-05

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [4]:
train, test = load_dataset("imdb", split=['train', 'test'])

Reusing dataset imdb (C:\Users\Fabrice\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)


In [5]:
train_text, train_label = train['text'], train['label']
test_text, test_label = test['text'], test['label']

train_text, val_text, train_label, val_label = train_test_split(train_text, train_label, test_size=.2)

In [6]:
print('train_text size:', len(train_text))
print('val_text size:  ', len(val_text))
print('test_text size: ', len(test_text))

train_text size: 20000
val_text size:   5000
test_text size:  25000


In [7]:
train_encodings = tokenizer(train_text, truncation=True, padding=True)
val_encodings = tokenizer(val_text, truncation=True, padding=True)
test_encodings = tokenizer(test_text, truncation=True, padding=True)

In [8]:
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
train_dataset = IMDbDataset(train_encodings, train_label)
val_dataset = IMDbDataset(val_encodings, val_label)
test_dataset = IMDbDataset(test_encodings, test_label)

In [9]:
class CustomTrainer(Trainer):
    """
    Use this Trainer for soft labels
    """
    def compute_loss(self, model, inputs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        return F.binary_cross_entropy_with_logits(logits, labels)
    
def custom_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    max_steps=40000,
    save_steps=5000,
    save_total_limit=1,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100
)

trainer = Trainer(
    model=model,                  
    args=training_args,           
    train_dataset=train_dataset,  
    eval_dataset=val_dataset,     
    compute_metrics=custom_metrics
)

In [12]:
trainer.train()

Step,Training Loss
1000,0.620448
2000,0.565645
3000,0.564673
4000,0.523062
5000,0.509332
6000,0.516319
7000,0.499748
8000,0.386437
9000,0.346559
10000,0.385806


TrainOutput(global_step=33335, training_loss=0.3647364188263365)

In [13]:
trainer.evaluate()

{'eval_loss': 0.5046678185462952,
 'eval_accuracy': 0.9056,
 'eval_f1': 0.9041040227549777,
 'eval_precision': 0.9004451639012545,
 'eval_recall': 0.9077927376580988,
 'epoch': 5.0}